In [ ]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../')

import constants
import extension_tools
import os

# import datasets
# import log_reg
# from dataproc import extract_wvs
# from dataproc import get_discharge_summaries
# from dataproc import concat_and_split
# from dataproc import build_vocab
# from dataproc import word_embeddings

# from nltk.tokenize import RegexpTokenizer
# import pandas as pd
# from tqdm import tqdm

We assume to start that you have prepared the MIMIC-III note files/data splits based on the CAML preprocessing notebook, available here:
https://github.com/jamesmullenbach/caml-mimic/blob/master/notebooks/dataproc_mimic_III.ipynb

and have the files in the following directory structure:

#### Intermediate step: write each note to its own file:

In [ ]:
#TOINSERT

### Step one: run cTAKES to perform concept annotation on the notefiles:

In [ ]:
#TOINSERT

### Step two: extract annotated concepts from the XMI files for each note and write to csv, by dev/test/train splits:

In [ ]:
#TOINSERT

### Step three: build the concept-text alignment matrices for each note:
#### This step also constructs a concept vocabulary file based on concepts with >3 occurrences (in TRAINING data).

#### NOTE: this takes some time to run**

In [ ]:
#example, for SNOMED only:
extension_tools.get_concept_text_alignment(inpt_file='/data/swiegreffe6/NEW_MIMIC/mimic3/train_full.csv', concepts_file='/data/swiegreffe6/NEW_MIMIC/patient_notes/concepts_train_ALL.csv', outpt_file='/data/swiegreffe6/NEW_MIMIC/extracted_concepts/SNOMED/train_patient_concepts_matrix_SNOMED.p', lst_terms=['SNOMEDCT_US'], dir_name='SNOMED')

#or as command, 
    python extension_tools get_concept_text_alignment --input-dir '/data/swiegreffe6/NEW_MIMIC/mimic3/train_full.csv' --concepts-file '/data/swiegreffe6/NEW_MIMIC/patient_notes/concepts_train_ALL.csv' --output-dir='/data/swiegreffe6/NEW_MIMIC/extracted_concepts/SNOMED/train_patient_concepts_matrix_SNOMED.p' --codes-list 'SNOMEDCT_US' --dir-name='SNOMED'

#also outputs the concept_children file to '/data/swiegreffe6/NEW_MIMIC/extracted_concepts/dir_name/concept_vocab_%s_children.txt' % dir_name (hardcoded)

In [4]:
#TODO: this step could use some more work, e.g. overlapping concepts, and hyphenated concepts that can't be mapped to any parent codes later
#consider some normalization here?

### Step four: build the child-to-parent code mappings based on the ontology resources we want to use:
#### NOTE this file only has to be created once (as long as complete ontologies included), because it's a dictionary for later lookup**

File directory needed to run this code:

    | constants.DATA_DIR
        | Multi_Level_CCS_2015
            | ccs_multi_dx_tool_2015.csv
            | ccs_multi_pr_tool_2015.csv
        | SnomedCT_USEditionRF2_PRODUCTION_20180301T183000Z 
            | Full
                | Terminology
                    | sct2_Relationship_Full_US1000124_20180301.txt

In [3]:
dirs_map = extension_tools.get_parent_trees()
#make sure constants.DATA_DIR points to directory holding subfolders of concept mappings keys.
#this method writes 'code_parents.p' to constants.concept_write_dir.

#TODOs/NOTE: this file contains/writes a 'lopsided' SNOMED hierarchy, where for each child we only consider one parent.
#We also have not included RXCLASS mapping file here (yet).

('LEN DIRS MAP before SNOMED add:', 19020)


100%|██████████| 5008688/5008688 [07:22<00:00, 11326.26it/s]
IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



### Next, we update the concept vocabulary to include the newly added codes

In [4]:
#Make sure path_to_vocab is the location of the (>3 occurrences) concept vocab generated from the training set.
#This outputs the updated concept vocabulary (plus the higher-level codes) to out_dir/concept_vocab.txt

path_to_vocab = os.path.join(concept_write_dir, 'train_meta_concepts.txt')
extension_tools.update_vocab(dirs_map, path_to_vocab, out_dir='/data/swiegreffe6/NEW_MIMIC/extracted_concepts/ICD9_RXNORM_SNOMED/ICD9_SNOMED_augmented_concept_vocab.txt')